# Daly Weights, inflow and outflow of ICD-10 - dalys: Panel creation

This script runs a `sql` code to create a panel data, based on the `case_records_current` table, where we have each `ICD-10` member change of status, and the link between the status change with change in daly weights. 
The calculation involves a lot of cleaning,   such as desconsidering impetous changes of status or *cancelled* icds status as the first status input, etc, we leave that for the sql code, and here is just the script to 
run it and export to the redshift.

Steps:

    - run this notebook to upload to Redshift;
    
    - use the new panel to make calculations, new tables, dashes...
***

## 0. Setup

In [3]:
#connection to redshift
from redshift_import_export.interfaces.data_interactor import DataInteractor
di = DataInteractor()
import requests


#standard packages
import warnings
warnings.filterwarnings('ignore')
import os
import collections
import pandas as pd
import numpy as np



#time packages
import datetime as dt

## 1. Import

In [4]:
df = di.redshift.run_sql_query(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/queries/daly_lower_bounds_weights_member_cid_panel_table.sql')
df.head(10)

DatabaseError: Execution failed on sql 'WITH 
---get the icd-10 registries at Alice
cdc_df AS (
    SELECT 
        crc.person_internal_code AS member_internal_code,
        crc.case_id,
        crc.sex,
        crc.age,
        crc.disease_code_value,
        crc.status,
        crc.added_at::timestamp as added_at,
        crc.severity
    FROM
        "public"."case_record_current" AS crc
    WHERE
        crc.status <> 'PENDING'
        AND crc.status <> 'CONTEMPLATING'
        AND crc.disease_code_type = 'CID_10'
),
cdc_df_distinct as(
    SELECT DISTINCT
        cdc_df.member_internal_code,
        cdc_df.case_id,
        cdc_df.added_at::timestamp as added_at,
        cdc_df.disease_code_value,
        cdc_df.status,
        cdc_df.severity
    FROM 
        cdc_df
    
    ), 
--FROM cte to cte6, we build a table that, for each case_id, we get
-- the order of updates. That way, we can compare a status change to
-- the previous status change and see if it is an improvement or not

## 3. Export

In [ ]:
di.redshift.insert_table(df, table_name='daly_lower_bounds_weights_member_cid_panel_table', if_exists='replace')